In [3]:
%reload_ext autoreload
%autoreload 2

# Libraries
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import random

from Autoencoder_class import *
from Autoencoder_functions import *
from data_functions import *

# # Seed fix
# seed = 10
# utils.set_seed(seed)
# random.seed(seed)


ModuleNotFoundError: No module named 'dill'

## Van der Pol

In [ ]:
# Simulation parameters
dt = 0.1  # time gap
simulation_time = 5 # simulation time (= length of each trajectory)
iteration_len = int(simulation_time/dt)
training_number = 500 # number of training data trajectory
testing_number = 100 # number of testing data trajectory (no validation set now)
Generate_Koopman_data = True # Load data if false

# Conformal prediction parameters - ignore these
CP_confidence = 0.99 # confidence bound for ellipsoid (= reachable set bound)
CP_sample_number = 1000

# Autoencoder & learning parameters
hidden_dimension = 20
batch_size = int(256)
lr = 0.001 # 1e-5
p = 20  # future time step for KAE training
num_epochs = 100
c1, c2, c3 = 1, 1, 1 # loss function parameters
c4 = 0.5 # loss function parameter for conformal part (ignore this)
temperature = 0.3 # parameter for conformal part (ignore this)

dynamics_model = 'vdp'
bias = False
Koopman_training_trajectory, Koopman_testing_trajectory, state_dimension, input_dimension, state_bounds = generate_dataset(dynamics_model, iteration_len, training_number, testing_number, simulation_time, bias, Generate_Koopman_data)

for i in range(0,training_number):
    plt.plot(Koopman_training_trajectory[0,0,i],Koopman_training_trajectory[1,0,i],'ro')
    plt.plot(Koopman_training_trajectory[0,:,i],Koopman_training_trajectory[1,:,i])

for i in range(0,testing_number):
    plt.plot(Koopman_testing_trajectory[0,0,i],Koopman_testing_trajectory[1,0,i],'bo')

In [ ]:
# KAE
testing_epochs = [10, 20, 30, 40, 50]

Conformal = False
KAE = KoopmanAutoencoder(state_dimension, hidden_dimension)
mean_error_history, var_error_history = train_KAE(KAE, state_dimension, hidden_dimension, batch_size, training_number, lr, 
              Koopman_training_trajectory, c1, c2, c3, c4, p, temperature, state_bounds, dynamics_model, testing_epochs,
              testing_number, testing_number, input_dimension, iteration_len, Koopman_testing_trajectory,
              CP_confidence, CP_sample_number, num_epochs, Conformal)

# KAE = torch.load("models/vdp_KAE_final"+str(num_epochs)+"_trj"+str(training_number)+".pkl", weights_only=False)
KAE_error = test_KAE(KAE, testing_number, testing_number, state_dimension, input_dimension, iteration_len, Koopman_testing_trajectory)

print(mean_error_history)
print(var_error_history)

print(np.mean(KAE_error))
print(np.var(KAE_error))